In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from ArV_NeuroSynth import ArV_NeuroSynth
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import scipy.io
import numpy as np # Import numpy with the alias np

mat = scipy.io.loadmat("A01T.mat")
data = mat['data']

for i in range(9):
    item = data[0, i]
    if isinstance(item, np.ndarray) and item.shape == (1, 1):
        value = item[0, 0]
    else:
        value = item
    print(f"Entry {i}: type={type(value)}, shape={getattr(value, 'shape', None)}")

# Corrected indentation for the second for loop
for i in range(9):
    struct = mat['data'][0, i][0, 0]
    print(f"Entry {i} field names:", struct.dtype.names)

Entry 0: type=<class 'numpy.void'>, shape=()
Entry 1: type=<class 'numpy.void'>, shape=()
Entry 2: type=<class 'numpy.void'>, shape=()
Entry 3: type=<class 'numpy.void'>, shape=()
Entry 4: type=<class 'numpy.void'>, shape=()
Entry 5: type=<class 'numpy.void'>, shape=()
Entry 6: type=<class 'numpy.void'>, shape=()
Entry 7: type=<class 'numpy.void'>, shape=()
Entry 8: type=<class 'numpy.void'>, shape=()
Entry 0 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 1 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 2 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 3 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 4 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 5 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifacts', 'gender', 'age')
Entry 6 field names: ('X', 'trial', 'y', 'fs', 'classes', 'artifact

In [ ]:
# Modify the load_bci_mat function to correctly access the nested EEG data
import scipy.io
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class EEGDataset(Dataset):
    def __init__(self, eeg_data, segment_length):
        self.eeg_data = eeg_data
        self.segment_length = segment_length
        # Ensure eeg_data is a PyTorch tensor
        if not isinstance(self.eeg_data, torch.Tensor):
             # If the data is float64 from scipy, convert to float32 for CUDA compatibility
             self.eeg_data = torch.tensor(self.eeg_data, dtype=torch.float32)


    def __len__(self):
        # Calculate the number of segments, potentially dropping the last partial one
        return self.eeg_data.shape[1] // self.segment_length

    def __getitem__(self, idx):
        # Extract a segment of the EEG data
        start_idx = idx * self.segment_length
        end_idx = start_idx + self.segment_length
        # Ensure the segment covers all channels (dimension 0)
        segment = self.eeg_data[:, start_idx:end_idx]
        return segment

def load_bci_mat(filepath, num_channels=4, segment_length=256, batch_size=32):
    """
    Loads EEG data from a .mat file, assuming a structure similar to BCI Competition IV dataset 2a.

    Args:
        filepath (str): Path to the .mat file.
        num_channels (int): Expected number of EEG channels.
        segment_length (int): The desired length of each EEG segment (time steps).
        batch_size (int): The batch size for the DataLoader.

    Returns:
        tuple: A tuple containing:
            - dataloader (DataLoader): PyTorch DataLoader for the EEG data.
            - eeg_tensor (torch.Tensor): The full loaded EEG data as a PyTorch tensor.
    """
    try:
        mat = scipy.io.loadmat(filepath, squeeze_me=True) # Use squeeze_me for easier access to nested arrays
    except FileNotFoundError:
        raise FileNotFoundError(f"Error: File not found at {filepath}")
    except Exception as e:
        raise IOError(f"Error loading .mat file: {e}")

    # Attempt to access the nested EEG data assuming it's in 'data' field,
    # and then potentially under a field like 'eeg' within the structured array.
    # Based on the user's print statements, 'data' is a 1x9 object array,
    # and the first element (data[0, 0]) is a struct.
    # We need to find the actual EEG data array within this struct.
    # We'll iterate through the first element's fields to find a likely candidate.
    data = None
    if 'data' in mat and mat['data'].size > 0:
        # Assuming the first element contains the structure with the EEG data
        first_item = mat['data'][0]
        if isinstance(first_item, np.ndarray) and first_item.dtype.names is not None:
            # Iterate through field names to find a large numerical array, likely the EEG data
            for name in first_item.dtype.names:
                 # Check if the field contains a numpy array
                field_data = first_item[name]
                if isinstance(field_data, np.ndarray) and field_data.ndim == 2 and field_data.shape[0] == num_channels:
                    data = field_data
                    print(f"Found potential EEG data under field: '{name}' with shape {data.shape}")
                    break # Assume the first suitable field is the EEG data

    if data is None:
        raise ValueError("Could not locate EEG data structure within the .mat file. "
                         "Checked for a 2D array with 'num_channels' rows within the 'data' structure fields.")

    # Transpose the data if needed to be [channels, time_steps]
    # Assuming the loaded data might be [time_steps, channels]
    if data.shape[0] != num_channels and data.shape[1] == num_channels:
        data = data.T
        print(f"Transposed data to shape: {data.shape}")
    elif data.shape[0] != num_channels:
         raise ValueError(f"EEG data has unexpected number of channels. Expected {num_channels}, found {data.shape[0]}.")


    # Create a PyTorch tensor and the dataset
    eeg_tensor = torch.tensor(data, dtype=torch.float32)
    eeg_dataset = EEGDataset(eeg_tensor, segment_length)

    # Create the DataLoader
    dataloader = DataLoader(eeg_dataset, batch_size=batch_size, shuffle=True)

    print(f"Created DataLoader with batch size {batch_size} and segment length {segment_length}.")
    print(f"Number of segments available: {len(eeg_dataset)}")


    return dataloader, eeg_tensor

In [ ]:
model = ArV_NeuroSynth(input_channels=4, seq_length=256, latent_dim=32, noise_dim=50).to(device)


In [ ]:
from ArV_NeuroSynth import compute_gradient_penalty
import torch.nn as nn

opt_vae = torch.optim.Adam(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=1e-4)
opt_g = torch.optim.Adam(model.G_latent.parameters(), lr=1e-4, betas=(0.5, 0.9))
opt_d = torch.optim.Adam(model.D_latent.parameters(), lr=1e-4, betas=(0.5, 0.9))

loss_fn = nn.MSELoss()
λ_gp = 10
β = 1e-5
epochs = 5

for epoch in range(epochs):
    model.train()
    total_vae_loss, total_d_loss, total_g_loss = 0, 0, 0
    for x_batch, in dataloader:
        x_batch = x_batch.to(device)

        # VAE
        x_recon, mu, logvar, z_real = model(x_batch, mode='vae')
        kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        vae_loss = loss_fn(x_recon, x_batch) + β * kl
        opt_vae.zero_grad(); vae_loss.backward(); opt_vae.step()
        total_vae_loss += vae_loss.item()

        # GAN - Discriminator
        for _ in range(5):
            z_fake = model(noise_for_g_latent=torch.randn(x_batch.size(0), model.noise_dim).to(device), mode='generate_latent_from_noise')
            d_real = model(z_real, mode='discriminate_latent')
            d_fake = model(z_fake, mode='discriminate_latent')
            gp = compute_gradient_penalty(model.D_latent, z_real, z_fake)
            d_loss = d_fake.mean() - d_real.mean() + λ_gp * gp
            opt_d.zero_grad(); d_loss.backward(); opt_d.step()
            total_d_loss += d_loss.item()

        # GAN - Generator
        z_fake = model(noise_for_g_latent=torch.randn(x_batch.size(0), model.noise_dim).to(device), mode='generate_latent_from_noise')
        g_loss = -model(z_fake, mode='discriminate_latent').mean()
        opt_g.zero_grad(); g_loss.backward(); opt_g.step()
        total_g_loss += g_loss.item()

    print(f"Epoch {epoch+1} | VAE: {total_vae_loss:.4f} | D: {total_d_loss:.4f} | G: {total_g_loss:.4f}")


NameError: name 'dataloader' is not defined

In [ ]:
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    x_real = next(iter(dataloader))[0][:1].to(device)
    x_recon, _, _, _ = model(x_real, mode='vae')

plt.figure(figsize=(12, 4))
for ch in range(x_real.shape[1]):
    plt.plot(x_real.cpu()[0, ch], label=f"Real Ch{ch}")
    plt.plot(x_recon.cpu()[0, ch], '--', label=f"Reconstructed Ch{ch}")
plt.legend(); plt.title("Real vs Reconstructed EEG"); plt.grid(); plt.tight_layout(); plt.show()


NameError: name 'model' is not defined